# CREATE DASHBOARD VISUALISATION 
---

## 1. Setup et data loading

In [1]:
import pandas as pd 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datetime

In [52]:
def millify(n):
    if n > 999:
        if n > 1e6-1:
            return f'{round(n/1e6,1)}M'
        return f'{round(n/1e3,1)}K'
    return n

In [51]:
sales = pd.read_csv('sales_summary.csv', parse_dates=[0])
dashboard_date = datetime.date(2019, 5, 23)

## 2. Création des filtres

In [190]:
# Dropdown choix de la date : 'Janvier → 1', Février → 2, 'Mars: 3', 'year → :'
month = [2]
# month = list(range(12))

filtred_sales = sales.loc[sales['Date'].dt.month.isin(month)]

In [191]:
# dropdown choix de la mesure : profit_2020, sales_2020
metric = 'sales_2020'  
target = 'sales_target' if metric == 'sales_2020' else 'profit_target'

## 3. Création des KPI

In [192]:
# progress
amount = filtred_sales[metric].sum()
progress = amount /  filtred_sales[target].sum()

In [193]:
print('amount:{}'.format(amount))
print('Objectif: {}'.format(progress))
print('score: {}'.format(score))

amount:3087003.9861963084
Objectif: 1.2190390443782895
score: 0.20785733206289048


### Progress bar

In [194]:
rest = 1 - progress if 1 - progress > 0 else 0
progress_color = 'green' if progress > 1 else 'red' 
values = [progress, rest]
colors = [progress_color, 'white']

# Use `hole` to create a donut-like pie chart
fig = go.Figure(
    go.Pie(
        values = values, 
        hole = .95, 
        marker_colors = colors
    )
)
fig.update_layout(
    showlegend = False,
    hovermode = False,
)
fig.update_traces(textinfo='none')
fig.update_layout(
    annotations = [
        dict(
            x=0.5,
            y=0.40,
            text="Objectif de Mars",
            showarrow=False,
            font=dict(
                size=30,
                color="#000000"
            )
        ),
        dict(
            x=0.5,
            y=0.55,
            text='{}%'.format(int(progress*100)),
            showarrow=False,
            font=dict(
                size=70,
                color=progress_color
            )
        )
    ]
)

### Amount of sales

In [195]:
target_goal = filtred_sales.loc[filtred_sales['Date'].dt.date <= dashboard_date, target].sum()
score = amount / target_goal -1

if score > 0:
    color = "green"
    text_score = '+ {}% ⬆︎'.format(int(score*100))
else:
    color = "red"
    text_score = '{}% ⬇︎'.format(int(score*100))   

# create a white pie
fig = go.Figure(go.Pie(values = [0,0]))
# add annotation
fig.update_layout(
    annotations = [
        dict(
            x=0.5,
            y=0.45,
            text="Chiffre d'Affaires ($)" if metric=="sales_2020" else "Bénéfices",
            showarrow=False,
            font=dict(
                size=30,
                color="#000000"
            )
        ),
        dict(
            x=0.5,
            y=0.60,
            text='{}'.format(millify(amount)),
            showarrow=False,
            font=dict(
                size=70,
                color="blue"
            )
        ),
        dict(
            x=0.5,
            y=0.35,
            text=text_score,
            showarrow=False,
#             bgcolor="#ff7f0e",
            font=dict(
                size=30,
                color=color
            )
        )
    ]
)

# 4. Bar chart par ville

In [9]:
# usefull function
def red_or_green(df):
    return df.apply(lambda x: 'green' if x >=1 else 'red')

In [10]:
city_sales = filtred_sales.groupby('City').sum()
percents = city_sales[metric] / city_sales[target]

# plot
fig = go.Figure([
    go.Bar(
        name = 'target', 
        y = city_sales.index, 
        x = city_sales[target], 
        marker_color='grey',
    ),
    go.Bar(
        name = metric, 
        y = city_sales.index, 
        x = city_sales[metric],
        text = percents,
        marker_color = red_or_green(percents),
        width = 0.5,
        
    )
])
fig.update_traces(texttemplate='%{text:.0%}', textposition='auto', selector=dict(name=metric))
fig.update_traces(orientation='h')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', barmode='overlay')

# Line plot par mois

- [ ] cumsum
- [ ] réfléchir au design

In [11]:
daily_sales = sales.groupby('Date').sum()
monthly_sales = daily_sales.resample('MS').sum()
percents = monthly_sales[metric] / monthly_sales[target]

# plot
fig = go.Figure([
    go.Bar(
        name = 'target', 
        x = monthly_sales.index, 
        y = monthly_sales[target], 
        marker_color='grey'
    ),
    go.Bar(
        name = metric, 
        x = monthly_sales.index, 
        y = monthly_sales[metric],
        text = percents,
        marker_color = red_or_green(percents),
        width = 0.6 *(1000*3600*24*22)
    )
])
fig.update_traces(texttemplate='%{text:.0%}', textposition='auto', selector=dict(name=metric))
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', barmode='overlay')

# Pie par categories 

In [12]:
from plotly.subplots import make_subplots

In [13]:
sales_per_cat = filtred_sales.groupby(['Cat', 'Date']).sum()
sales_per_cat.reset_index(0, inplace=True)

categories = (filtred_sales['Cat'].unique())

In [14]:

fig = make_subplots(rows=1, cols=len(categories), 
                    specs = [[{"type": "domain"}]*len(categories)])

for i,cat in enumerate(categories):
    df = sales_per_cat.loc[sales_per_cat['Cat'] == cat]
    progress = df[metric].sum() / df[target].sum()
    rest = 1 - progress if 1 - progress > 0 else 0
    fig.append_trace(
        go.Pie(
            values = [progress, rest], 
            hole = .95,
            title = '{}'.format(cat),
            textinfo='none'
    ), row=1, col=i+1)

    
fig.update_layout(
    showlegend = False,
    hovermode = False,
    font_size = 15,
)
fig.show()